# Step 1. Extract Feature

In [1]:
import torch
import torchvision
image = torch.zeros((1, 3, 800, 800)).float()

bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) # [y1, x1, y2, x2] format
labels = torch.LongTensor([6, 8]) # 0 represents background
sub_sample = 16

In [2]:
labels

tensor([6, 8])

In [3]:
bbox

tensor([[ 20.,  30., 400., 500.],
        [300., 400., 500., 600.]])

In [4]:
dummy_img = torch.zeros((1, 3, 800, 800)).float()
print(dummy_img.shape)

torch.Size([1, 3, 800, 800])


In [ ]:
model = torchvision.models.vgg16(pretrained=True)
fe = list(model.features)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/albertyu/.torch/models/vgg16-397923af.pth
61095936it [10:57, 868.12it/s]   

In [ ]:
print(model.features)

In [10]:
import numpy as np
ratios = [0.5, 1, 2]
anchor_scales = [8, 16, 32]

anchor_base = np.zeros((len(ratios) * len(anchor_scales), 4), dtype=np.float32)

print(anchor_base)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [15]:
ctr_y = sub_sample / 2.
ctr_x = sub_sample / 2.

for i in range(len(ratios)):
    for j in range(len(anchor_scales)):
        h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
        w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])

        index = i * len(anchor_scales) + j

        anchor_base[index, 0] = ctr_y - h / 2.
        anchor_base[index, 1] = ctr_x - w / 2.
        anchor_base[index, 2] = ctr_y + h / 2.
        anchor_base[index, 3] = ctr_x + w / 2.

8.0 8.0


In [83]:
fe_size = (800//16)
ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)

In [86]:
ctr_x

array([ 16,  32,  48,  64,  80,  96, 112, 128, 144, 160, 176, 192, 208,
       224, 240, 256, 272, 288, 304, 320, 336, 352, 368, 384, 400, 416,
       432, 448, 464, 480, 496, 512, 528, 544, 560, 576, 592, 608, 624,
       640, 656, 672, 688, 704, 720, 736, 752, 768, 784, 800])

In [87]:
index = 0
ctr = np.zeros((len(ctr_x)*len(ctr_y), 2))

for x in range(len(ctr_x)):
    for y in range(len(ctr_y)):
        ctr[index, 1] = ctr_x[x] - 8
        ctr[index, 0] = ctr_y[y] - 8
        index = index + 1

In [89]:
ctr.shape

(2500, 2)

In [90]:
anchors = np.zeros((fe_size * fe_size * 9, 4))
index = 0
for c in ctr:
    ctr_y, ctr_x = c
    for i in range(len(ratios)):
        for j in range(len(anchor_scales)):
            h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
            w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])
            anchors[index, 0] = ctr_y - h / 2.
            anchors[index, 1] = ctr_x - w / 2.
            anchors[index, 2] = ctr_y + h / 2.
            anchors[index, 3] = ctr_x + w / 2.
            index += 1

In [91]:
anchors.shape

(22500, 4)

In [92]:
index_inside = np.where(
        (anchors[:, 0] >= 0) &
        (anchors[:, 1] >= 0) &
        (anchors[:, 2] <= 800) &
        (anchors[:, 3] <= 800)
    )[0]
print(index_inside.shape)

(8940,)


In [95]:
label = np.empty((len(index_inside), ), dtype=np.int32)
label.fill(-1)

In [132]:
index_inside

array([ 1410,  1419,  1428, ..., 21075, 21084, 21093])

In [99]:
valid_anchor_boxes = anchors[index_inside]
print(valid_anchor_boxes.shape)

(8940, 4)


In [100]:
bbox = np.asarray([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=np.float32) # [y1, x1, y2, x2] format
labels = np.asarray([6, 8], dtype=np.int8) # 0 represents background

In [101]:

#!/usr/bin/env python
# encoding: utf-8
 
 
 
def compute_iou(rec1, rec2):
    """
    computing IoU
    :param rec1: (y0, x0, y1, x1), which reflects
            (top, left, bottom, right)
    :param rec2: (y0, x0, y1, x1)
    :return: scala value of IoU
    """
    # computing area of each rectangles
    S_rec1 = (rec1[2] - rec1[0]) * (rec1[3] - rec1[1])
    S_rec2 = (rec2[2] - rec2[0]) * (rec2[3] - rec2[1])
 
    # computing the sum_area
    sum_area = S_rec1 + S_rec2
 
    # find the each edge of intersect rectangle
    left_line = max(rec1[1], rec2[1])
    right_line = min(rec1[3], rec2[3])
    top_line = max(rec1[0], rec2[0])
    bottom_line = min(rec1[2], rec2[2])
 
    # judge if there is an intersect
    if left_line >= right_line or top_line >= bottom_line:
        return 0
    else:
        intersect = (right_line - left_line) * (bottom_line - top_line)
        return (intersect / (sum_area - intersect))*1.0
 
 
if __name__=='__main__':
    rect1 = (661, 27, 679, 47)
    # (top, left, bottom, right)
    rect2 = (662, 27, 682, 47)
    iou = compute_iou(rect1, rect2)


In [104]:
valid_anchor_boxes = anchors[index_inside]
print(valid_anchor_boxes.shape)
#Out = (8940, 4)

(8940, 4)


In [107]:
ious = np.empty((len(valid_anchor_boxes), 2), dtype=np.float32)
ious.fill(0)
print(bbox)
for num1, i in enumerate(valid_anchor_boxes):
    ya1, xa1, ya2, xa2 = i  
    anchor_area = (ya2 - ya1) * (xa2 - xa1)
    for num2, j in enumerate(bbox):
        yb1, xb1, yb2, xb2 = j
        box_area = (yb2- yb1) * (xb2 - xb1)
        inter_x1 = max([xb1, xa1])
        inter_y1 = max([yb1, ya1])
        inter_x2 = min([xb2, xa2])
        inter_y2 = min([yb2, ya2])
        if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
            iter_area = (inter_y2 - inter_y1) * \
(inter_x2 - inter_x1)
            iou = iter_area / \
(anchor_area+ box_area - iter_area)            
        else:
            iou = 0.
        ious[num1, num2] = iou
print(ious.shape)
#Out: [22500, 2]

[[ 20.  30. 400. 500.]
 [300. 400. 500. 600.]]
(8940, 2)


In [108]:
gt_argmax_ious = ious.argmax(axis=0)
print(gt_argmax_ious)
gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]
print(gt_max_ious)
# Out:
# [2262 5620]
# [0.68130493 0.61035156]

[2262 5620]
[0.68130493 0.61035156]


In [110]:
argmax_ious = ious.argmax(axis=1)
print(argmax_ious.shape)
print(argmax_ious)
max_ious = ious[np.arange(len(index_inside)), argmax_ious]
print(max_ious)

(8940,)
[0 0 0 ... 0 0 0]
[0.06811669 0.07083762 0.07083762 ... 0.         0.         0.        ]


In [111]:
gt_argmax_ious = np.where(ious == gt_max_ious)[0]
print(gt_argmax_ious)


[2262 2508 5620 5628 5636 5644 5866 5874 5882 5890 6112 6120 6128 6136
 6358 6366 6374 6382]


In [113]:
ious.shape

(8940, 2)

In [121]:
ious[gt_argmax_ious]

array([[0.68130493, 0.08628624],
       [0.68130493, 0.09892923],
       [0.10757449, 0.61035156],
       [0.09517316, 0.61035156],
       [0.08304646, 0.61035156],
       [0.07118537, 0.61035156],
       [0.09548767, 0.61035156],
       [0.08458613, 0.61035156],
       [0.07389943, 0.61035156],
       [0.06342126, 0.61035156],
       [0.0836618 , 0.61035156],
       [0.07420184, 0.61035156],
       [0.06490561, 0.61035156],
       [0.0557689 , 0.61035156],
       [0.07208853, 0.61035156],
       [0.0640145 , 0.61035156],
       [0.05606118, 0.61035156],
       [0.04822588, 0.61035156]], dtype=float32)

In [124]:
gt_argmax_ious

array([2262, 2508, 5620, 5628, 5636, 5644, 5866, 5874, 5882, 5890, 6112,
       6120, 6128, 6136, 6358, 6366, 6374, 6382])

In [125]:
pos_iou_threshold  = 0.7
neg_iou_threshold = 0.3

In [126]:
label[max_ious < neg_iou_threshold] = 0

In [127]:
label[gt_argmax_ious] = 1

In [130]:
label

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [133]:
label[max_ious >= pos_iou_threshold] = 1

In [134]:
label


array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [135]:
pos_ratio = 0.5
n_sample = 256

In [136]:
n_pos = pos_ratio * n_sample

In [137]:
n_pos

128.0

In [138]:
pos_index = np.where(label == 1)[0]
if len(pos_index) > n_pos:
    disable_index = np.random.choice(pos_index, size=(len(pos_index) - n_pos), replace=False)
    label[disable_index] = -1

In [141]:
n_neg = n_sample * np.sum(label == 1)
neg_index = np.where(label == 0)[0]
if len(neg_index) > n_neg:
    disable_index = np.random.choice(neg_index, size=(len(neg_index) - n_neg), replace = False)
    label[disable_index] = -1

In [145]:
 n_neg

4608

In [148]:
len(neg_index) - n_neg

3082

In [154]:
max_iou_bbox = bbox[argmax_ious]
print(max_iou_bbox)

[[ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]
 ...
 [ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]]


In [158]:
height = valid_anchor_boxes[:, 2] - valid_anchor_boxes[:, 0]
width = valid_anchor_boxes[:, 3] - valid_anchor_boxes[:, 1]
ctr_y = valid_anchor_boxes[:, 0] + 0.5 * height
ctr_x = valid_anchor_boxes[:, 1] + 0.5 * width

In [159]:
base_height = max_iou_bbox[:, 2] - max_iou_bbox[:, 0]
base_width = max_iou_bbox[:, 3] - max_iou_bbox[:, 1]
base_ctr_y = max_iou_bbox[:, 0] + 0.5 * base_height
base_ctr_x = max_iou_bbox[:, 1] + 0.5 * base_width

In [160]:
eps = np.finfo(height.dtype).eps

In [165]:
height = np.maximum(height, eps)
width = np.maximum(width, eps)

In [166]:
height

array([181.01933598, 181.01933598, 181.01933598, ..., 181.01933598,
       181.01933598, 181.01933598])

In [168]:
dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)

In [169]:
anchor_locs = np.vstack((dy, dx, dh, dw)).transpose()
print(anchor_locs)

[[ 0.5855728   2.30914558  0.7415674   1.64727602]
 [ 0.49718446  2.30914558  0.7415674   1.64727602]
 [ 0.40879611  2.30914558  0.7415674   1.64727602]
 ...
 [-2.50801936 -5.29225232  0.7415674   1.64727602]
 [-2.59640771 -5.29225232  0.7415674   1.64727602]
 [-2.68479606 -5.29225232  0.7415674   1.64727602]]


In [171]:
anchor_labels = np.empty((len(anchors),), dtype=label.dtype)
anchor_labels.fill(-1)
anchor_labels[index_inside] = label

In [175]:
anchor_locations = np.empty((len(anchors),) + anchors.shape[1:], dtype=anchor_locs.dtype)
anchor_locations.fill(0)
anchor_locations[index_inside, :] = anchor_locs

In [176]:
anchor_locations

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [177]:
import torch.nn as nn
mid_channels = 512
in_channels = 512 # depends on the output feature map. in vgg 16 it is equal to 512
n_anchor = 9 # Number of anchors at each location
conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
reg_layer = nn.Conv2d(mid_channels, n_anchor *4, 1, 1, 0)
cls_layer = nn.Conv2d(mid_channels, n_anchor *2, 1, 1, 0)
## I will be going to use softmax here. you can equally use sigmoid if u replace 2 with 1.

In [179]:
# conv sliding layer
conv1.weight.data.normal_(0, 0.01)
conv1.bias.data.zero_()
# Regression layer
reg_layer.weight.data.normal_(0, 0.01)
reg_layer.bias.data.zero_()
# classification layer
cls_layer.weight.data.normal_(0, 0.01)
cls_layer.bias.data.zero_()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [180]:
x = conv1(out_map) # out_map is obtained in section 1
pred_anchor_locs = reg_layer(x)
pred_cls_scores = cls_layer(x)


NameError: name 'out_map' is not defined